In [0]:
import pandas as pd
import numpy as np
import sqlite3
build_data_url = 'https://raw.githubusercontent.com/med-cabinet-5/data-science/master/data/cannabis_2_postgresql.csv'
build_data = pd.read_csv(build_data_url)
build_data = build_data.fillna("")
build_data.head(10)
build_data.iloc[2361]
effects_list = []
for i in range(len(build_data)):
    for effect in build_data['Effects'].iloc[i].split(','):
        #print(effect, i)
        if effect not in effects_list:
            effects_list.append(effect)
print(f'There are {len(effects_list)} unique reported effects among the {len(build_data)} strains.')
flavors_list = []
for i in range(len(build_data)):
    for effect in build_data['Flavor'].iloc[i].split(','):
        #print(effect, i)
        if effect not in flavors_list:
            flavors_list.append(effect)
print(f'There are {len(flavors_list)} unique reported flavors among the {len(build_data)} strains.')
ailments_list = []
for i in range(len(build_data)):
    for effect in build_data['Ailment'].iloc[i].split(','):
        #print(effect, i)
        if effect not in ailments_list:
            ailments_list.append(effect)
print(f'There are {len(ailments_list)} unique reported ailments among the {len(build_data)} strains.')
strain_types = []
for i in range(len(build_data)):
    for effect in build_data['Type'].iloc[i].split(','):
        #print(effect, i)
        if effect not in strain_types:
            strain_types.append(effect)
print(f'There are {len(strain_types)} unique reported strain types among the {len(build_data)} strains.')

There are 36 unique reported effects among the 2362 strains.
There are 98 unique reported flavors among the 2362 strains.
There are 25 unique reported ailments among the 2362 strains.
There are 5 unique reported strain types among the 2362 strains.


In [0]:
import sqlite3
db = '/content/db.sqlite3'
sl_conn = sqlite3.connect(db)
sl_curs = sl_conn.cursor()
build_data[['Strain', 'Type', 'Effects', 'Ailment', 'Flavor', 'Description']].to_sql('strains_table', sl_conn)
sl_conn.commit()

In [0]:
query = "SELECT Effects FROM strains_table LIMIT 10"

sl_curs.execute(query).fetchall()

[('Focused, Happy, Giggly, Euphoric, Relaxed, Hungry, Energetic, Creative, Tingly, Sleepy',),
 ('Focused, Euphoric, Relaxed, Energetic, Creative, Tingly',),
 ('Happy, Relaxed, Energetic, Creative, Anxious, Uplifted, Mouth, Dry',),
 ('Relaxed, Hungry, Creative, Uplifted, Tingly',),
 ('Happy, Euphoric, Relaxed, Uplifted, Talkative',),
 ('',),
 ('Happy, Euphoric, Relaxed, Hungry, Uplifted',),
 ('Focused, Happy, Giggly, Euphoric, Relaxed, Hungry, Anxious, Uplifted, Mouth, Dry',),
 ('Happy, Focused, Relaxed, Uplifted, Mouth, Dry, Tingly, Sleepy, Talkative',),
 ('Happy, Uplifted, Talkative, Energetic',)]

In [0]:
sl_conn.commit()

In [0]:
# build_data.head()
# build_data[['Strain', 'Type', 'Effects', 'Ailment', 'Flavor', 'Description']]

In [0]:
!pip install psycopg2-binary

In [0]:
import psycopg2

In [0]:
# dbname = ''
# user = ''
# password = ''  # Don't commit or share this for security purposes!
# host = ''  # Port should be included or default

In [0]:
pg_conn = psycopg2.connect(dbname=dbname, user=user,
                           password=password, host=host)

In [0]:
pg_conn

<connection object at 0x7fced5c19b40; dsn: 'user=nhworfxj password=xxx dbname=nhworfxj host=balarama.db.elephantsql.com', closed: 0>

In [0]:
pg_curs = pg_conn.cursor()

In [0]:
# Our goal - copy the characters table from SQLite to PostgreSQL using Python
# Step 1 - E=Extract: Get the Characters

query = "SELECT * FROM strains_table"

strains = sl_curs.execute(query).fetchall()

In [0]:
strains[:5]

[(0,
  'One To One',
  'Hybrid',
  'Focused, Happy, Giggly, Euphoric, Relaxed, Hungry, Energetic, Creative, Tingly, Sleepy',
  '  ',
  'Floral, Pungent, Earthy, Wood, Pepper, Diesel, Pine',
  'Named for its equal balance of THC and CBD, One to One by CBD Seeds is a stabilized hybrid bred from Amnesia Haze and a high-CBD landrace strain. Its nuanced citrus notes come out strong from the first opening of the jar until the final exhale. With effects that embrace both the mind and body, One to One delivers blissful relaxation that encourages a positive mood. One to One finishes its flowering cycle after 8 to 9 weeks, or in September for outdoor gardens. Or, comprised of 70 percent indica genetics, is an Israeli strain from the medical cannabis supplier Tikun Olam. Primarily designed for patients with sleep disorders, Or also remedies an array of other symptoms including nausea, pain, inflammation, indigestion, and appetite loss. The One, originally bred by\xa0Clackamas Coots or CC, is a pe

In [0]:
len(strains)

2362

In [0]:
# Step 2 - Transform
# In this case, we don't actually want/need to change much
# Because we want to keep all the data
# And we're going from SQL to SQL

# But what do we need to be able to load into PostgreSQL?
# We need to make a new table with the appropriate schema

# What was the old schema? We can get at this with SQLite internals
sl_curs.execute('PRAGMA table_info(strains_table);').fetchall()

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'Strain', 'TEXT', 0, None, 0),
 (2, 'Type', 'TEXT', 0, None, 0),
 (3, 'Effects', 'TEXT', 0, None, 0),
 (4, 'Ailment', 'TEXT', 0, None, 0),
 (5, 'Flavor', 'TEXT', 0, None, 0),
 (6, 'Description', 'TEXT', 0, None, 0)]

In [0]:
# https://www.postgresql.org/docs/current/sql-createtable.html

create_strains_table = """
CREATE TABLE strains_table (
  index SERIAL PRIMARY KEY,
  Strain VARCHAR(30),
  Type VARCHAR(30),
  Effects VARCHAR(120),
  Ailment VARCHAR(80),
  Flavor VARCHAR(80),
  Description VARCHAR(2000)
);
"""

In [0]:
pg_curs.execute(create_strains_table)

In [0]:
pg_conn.commit()

In [0]:
# We can query tables if we want to check
# This is a clever optional thing, showing postgresql internals
show_tables = """
SELECT
   *
FROM
   pg_catalog.pg_tables
WHERE
   schemaname != 'pg_catalog'
AND schemaname != 'information_schema';
"""
pg_curs.execute(show_tables)
pg_curs.fetchall()

[('public', 'strains_table', 'nhworfxj', None, True, False, False, False)]

In [0]:
strains[0]

(0,
 'One To One',
 'Hybrid',
 'Focused, Happy, Giggly, Euphoric, Relaxed, Hungry, Energetic, Creative, Tingly, Sleepy',
 '  ',
 'Floral, Pungent, Earthy, Wood, Pepper, Diesel, Pine',
 'Named for its equal balance of THC and CBD, One to One by CBD Seeds is a stabilized hybrid bred from Amnesia Haze and a high-CBD landrace strain. Its nuanced citrus notes come out strong from the first opening of the jar until the final exhale. With effects that embrace both the mind and body, One to One delivers blissful relaxation that encourages a positive mood. One to One finishes its flowering cycle after 8 to 9 weeks, or in September for outdoor gardens. Or, comprised of 70 percent indica genetics, is an Israeli strain from the medical cannabis supplier Tikun Olam. Primarily designed for patients with sleep disorders, Or also remedies an array of other symptoms including nausea, pain, inflammation, indigestion, and appetite loss. The One, originally bred by\xa0Clackamas Coots or CC, is a perfectly

In [0]:
example_insert = """
INSERT INTO strains_table
(Strain, Type, Effects, Ailment, Flavor, Description)
VALUES """ + str(strains[1][1:]) + ";"

print(example_insert)


INSERT INTO strains_table
(Strain, Type, Effects, Ailment, Flavor, Description)
VALUES ('100 Og', 'Hybrid', 'Focused, Euphoric, Relaxed, Energetic, Creative, Tingly', 'Depression', 'Earthy, Sweet, Citrus', '<p>This strain is named after its high price in its Hollywood home. As a 50/50 hybrid of indica and sativa, $100 OG does a great job offering pain relief with an alert, cerebral high.</p> $100 OG is a 50/50 hybrid strain that packs a strong punch. The name supposedly refers to both its strength and high price when it first started showing up in Hollywood. As a plant, $100 OG tends to produce large dark green buds with few stems. Users report a strong body effect of an indica for pain relief with the more alert, cerebral feeling thanks to its sativa side.');


In [0]:
# How do we do this for all characters? Loops!
for strain in strains:
  insert_strain = """
    INSERT INTO strains_table
    (Strain, Type, Effects, Ailment, Flavor, Description)
    VALUES """ + str(strain[1:]) + ";"
  pg_curs.execute(insert_strain)

In [0]:
pg_conn.commit()